In [27]:
import sys
import subprocess
import importlib.util

print("Checking dependencies...")
required = ['pydicom', 'tkinter', 'matplotlib']

for name in required:
    if spec := importlib.util.find_spec(name) is None:
        subprocess.check_call([sys.esys.executable, '-m', 'pip', 'install', '--upgrade'] + name)

from pydicom import dcmread
from matplotlib.figure import Figure
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import filedialog
from tkinter.ttk import *

root = tk.Tk()

root.title("Scrollable Dose Map DICOM Viewer")
root.minsize(750,450)

filename = tk.StringVar(root)
e1 = Entry(root, textvariable=filename)
e1.grid(row=1, column=0, sticky=W, pady=2)
e1.config(state="readonly")

def browse():
    f = filedialog.askopenfilename()
    filename.set(f)

b1 = Button(root, text="Select Dose File", command=browse)
b1.grid(row=0, column=0, sticky=W, pady=2)

fig = Figure(figsize=(8,6), dpi=100)
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().grid(row=0, column=1, columnspan=3, rowspan=3, padx=5, pady=5)
canvas.draw()

slices, rows, cols = [0, 0, 0]
ind = slices//2
data = None
im = None
    
def onscroll(event):
    global ind, slices, ax, im
    if event.button == 'up':
        ind = (ind + 1) % slices
    else:
        ind = (ind - 1) % slices
    im.set_data(data[ind, :, :])
    ax.set_ylabel("Slice %s" % ind)
    im.axes.figure.canvas.draw()
      
fig.canvas.mpl_connect('scroll_event', onscroll)
    
def plot_dcm():
    global slices, rows, cols, ind, im, ax, data
    ds = dcmread(e1.get())
    data = ds.pixel_array
    slices, rows, cols = data.shape
    ind = slices//2
    im = ax.imshow(data[ind, :, :])
    canvas.draw()
    
b2 = Button(root, text="Plot", command=plot_dcm)
b2.grid(row=2, column=0, sticky=W, pady=2)

root.mainloop()

Checking dependencies...


In [26]:
!pyinstaller DoseView.py

2483 INFO: PyInstaller: 6.11.1, contrib hooks: 2024.10
2483 INFO: Python: 3.9.7 (conda)
2509 INFO: Platform: Windows-10-10.0.19045-SP0
2509 INFO: Python environment: C:\Users\gia3\Anaconda3
2510 INFO: wrote C:\Users\gia3\Desktop\DoseView.spec
2521 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\gia3\\Anaconda3\\Scripts\\pyinstaller.exe',
 'C:\\Users\\gia3\\Anaconda3\\python39.zip',
 'C:\\Users\\gia3\\Anaconda3\\DLLs',
 'C:\\Users\\gia3\\Anaconda3\\lib',
 'C:\\Users\\gia3\\Anaconda3',
 'C:\\Users\\gia3\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\gia3\\Anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg',
 'C:\\Users\\gia3\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\gia3\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\gia3\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\gia3\\Desktop']
5246 INFO: checking Analysis
5246 INFO: Building Analysis because Analysis-00.toc is non existent
5246 INFO: Running Analysis Analysis-00.toc
5246 INFO: Target bytecode